# NYC taxi time prediction

This notebook use the NYC taxi time dataset from https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data.
The dataset contains 1 458 644 rows, which is a quite big dataset for manipulation. This dataset has 11 columns:
* id: a unique identifier for each trip
* vendor_id: a code indicating the provider associated with the trip record
* pickup_datetime: data and time when the meter was engaged
* dropoff_datetime: date and time when the meter was disengaged
* passenger_count: the number of passengers in the vehicle (driver entered value)
* pickup_longitude: the longitude where the meter was engaged
* pickup_latitude: the latitude where the meter was engaged
* dropoff_longitude: the longitude where the meter was disengaged
* dropoff_latitude: the latitude where the meter was disengaged
* store_and_fwd_flag: whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server
* trip_duration: duration of trip in seconds

We will use greenplumpython and follow the data science cycle (Data Exploration - Data Preparation - Data Modeling - Model Evaluation - Model Deployment) to train a Linear Regression model to predict the taxi time using this dataset.

## Connection to database
The use of greenplumpython is quite easy, you just have to install it with pip install, and import it at the beginning of the notebook.
Then, you can do `gp.database()` to connect to the database.


In [1]:
import greenplumpython as gp

In [2]:
db = gp.database(host="localhost", dbname="postgres")

## Data Exploration

The dataset is stored in the greenplum table `nyc.train`, to access it, you can do `db.open_table("nyc.train")` to create a `Table` named nyc.


In [3]:
nyc = db.open_table("nyc.train")

To view the first 5 rows of the `Table` nyc ordered by id,

In [4]:
nyc.order_by("id")[:5]

id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_duration_log
id0000001,2,2016-06-14T10:43:10,2016-06-14T11:01:35,1,-74.01392364501953,40.713924407958984,-73.97164916992188,40.74640655517578,N,1105,3.0433622780211294
id0000003,2,2016-03-16T10:39:55,2016-03-16T10:57:21,5,-73.99075317382812,40.7547607421875,-74.00711822509766,40.74153137207031,N,1046,3.0195316845312554
id0000005,2,2016-04-25T09:50:48,2016-04-25T09:56:56,1,-73.95310974121094,40.798580169677734,-73.96817779541016,40.800270080566406,N,368,2.5658478186735176
id0000008,1,2016-06-15T09:57:05,2016-06-15T10:02:08,1,-74.00962829589844,40.724761962890625,-74.015869140625,40.715484619140625,N,303,2.481442628502305
id0000009,1,2016-05-08T01:43:11,2016-05-08T01:52:18,1,-73.98799133300781,40.7598991394043,-73.95968627929688,40.79850387573242,N,547,2.737987326333431


## Data Preparation

Create a new `Table` nyc_clean which contains rows of nyc where passenger_count <> 0

In [5]:
nyc_clean = nyc[lambda t: t["passenger_count"] != 0] # Eliminate record with 0 passenger on board

In [6]:
nyc_clean[:5]

id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,trip_duration_log
id3504673,2,2016-04-06T19:32:31,2016-04-06T19:39:40,1,-74.01004028320312,40.719970703125,-74.01226806640625,40.70671844482422,N,429,2.6324572921847245
id1324603,2,2016-05-21T07:54:58,2016-05-21T08:20:49,1,-73.96927642822266,40.79777908325195,-73.92247009277344,40.76055908203125,N,1551,3.190611797813605
id0012891,2,2016-03-10T21:45:01,2016-03-10T22:05:26,1,-73.98104858398438,40.74433898925781,-73.9729995727539,40.78998947143555,N,1225,3.0881360887005513
id1436371,2,2016-05-10T22:08:41,2016-05-10T22:29:55,1,-73.98265075683594,40.76383972167969,-74.00222778320312,40.73299026489258,N,1274,3.1051694279993316
id1187965,2,2016-02-19T09:52:46,2016-02-19T10:11:20,2,-73.96298217773438,40.75667953491211,-73.98440551757812,40.760719299316406,N,1114,3.04688519083771


Get access to two defined functions in Greenplum to help us extract the day of week, the hour and the month from column pickuo_datetime and add these three new columns calling `assign()`.

In [7]:
to_char = gp.function("To_Char")

In [8]:
date_part = gp.function("date_part")

In [9]:
nyc_clean = nyc_clean.assign(
    day_of_week=lambda t: to_char(t["pickup_datetime"], 'Day'), # Extract Day of week from pickup_datetime
    hour_of_the_day=lambda t: date_part('hour', t["pickup_datetime"]), # Extract hour from pickup_datetime
    month=lambda t: date_part('month', t["pickup_datetime"]) # Extract month from pickup_datetime
)

In [10]:
nyc_clean[["pickup_datetime", "day_of_week", "hour_of_the_day", "month"]][:5]

pickup_datetime,day_of_week,hour_of_the_day,month
2016-06-12T00:43:35,Sunday,0,6
2016-01-19T11:35:24,Tuesday,11,1
2016-03-26T13:30:55,Saturday,13,3
2016-05-27T23:12:23,Friday,23,5
2016-06-01T20:58:29,Wednesday,20,6


We can not only use defined function in database, but also combine python user defined function with our table.
Here, we create two functions: one to convert store_switch_flag to integer and the other convert day of week to int.

In [11]:
@gp.create_function
def switch_flag_int(flag: str) -> int:
    if flag == 'N':
        return 0
    return 1

@gp.create_function
def switch_day_int(day: str) -> int:
    day_d = day.strip()
    if day_d == 'Monday':
        return 1
    elif day_d == 'Tuesday':
        return 2
    elif day_d == 'Wednesday':
        return 3
    elif day_d == 'Thursday':
        return 4
    elif day_d == 'Friday':
        return 5
    elif day_d == 'Saturday':
        return 6
    else:
        return 7

Let's call assign() with these two functions above to extend the table nyc_clean with two new columns.

In [12]:
nyc_clean = nyc_clean.assign(
    store_and_fwd_flag_int=lambda t: switch_flag_int(t["store_and_fwd_flag"]), # Transform store_and_flag to binary int
    day_of_week_int=lambda t: switch_day_int(t["day_of_week"])) # Transform day_of_week to int

In [13]:
nyc_clean[["store_and_fwd_flag", "store_and_fwd_flag_int", "day_of_week", "day_of_week_int"]][:5]

store_and_fwd_flag,store_and_fwd_flag_int,day_of_week,day_of_week_int
N,0,Monday,1
N,0,Saturday,6
N,0,Friday,5
N,0,Sunday,7
N,0,Monday,1


The distance of trip influence a lot the time of duration, so here we create a function to compute the haversine distance between two geological glopes.

In [14]:
@gp.create_function
def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    import math

    # distance between latitudes
    # and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    # convert to radians
    llat1 = (lat1) * math.pi / 180.0
    llat2 = (lat2) * math.pi / 180.0

    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(llat1) * math.cos(llat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

In [15]:
nyc_clean = nyc_clean.assign(
    distance=lambda t: haversine(
                        t["pickup_latitude"],
                        t["pickup_longitude"],
                        t["dropoff_latitude"],
                        t["dropoff_longitude"],
    )
) # Compute distance between start and end of trip

In [16]:
nyc_clean[["distance"]][:5]

distance
1.485498422771006
5.71498063078935
5.121161562140607
3.80613939487619
1.8594830202295125


nyc_clean = nyc_clean[[
                "vendor_id",
                "passenger_count",
                "store_and_fwd_flag_int",
                "trip_duration",
                "day_of_week_int",
                "hour_of_the_day",
                "month",
                "distance",
]

After obtained our prepared data, let't save the data of `Table` nyc_clean into a new `Table`

In [18]:
db.execute("DROP TABLE IF EXISTS nyc_clean", has_results=False)

In [19]:
nyc_clean = nyc_clean.save_into("nyc_clean") # Save the clean prepared dataset to Greenplum

In [20]:
nyc_clean[:5]

vendor_id,passenger_count,store_and_fwd_flag_int,trip_duration,day_of_week_int,hour_of_the_day,month,distance
2,1,0,429,3,19,4,1.485498422771006
2,1,0,1551,6,7,5,5.71498063078935
2,1,0,1225,4,21,3,5.121161562140607
2,1,0,1274,2,22,5,3.80613939487619
2,2,0,1114,5,9,2,1.8594830202295125


### Train Test split
Let's split our cleaned data to train and test set.

In [21]:
db.execute("""
    DROP TABLE IF EXISTS temp_nyc_label;
    CREATE TEMP TABLE temp_nyc_label AS
        (SELECT *, random() AS __samp_out_label FROM nyc_clean);

    DROP TABLE IF EXISTS train_percentile_disc;
    CREATE TEMP TABLE train_percentile_disc AS
        (SELECT percentile_disc(0.8) within GROUP (ORDER BY __samp_out_label) AS __samp_out_label
        FROM temp_nyc_label);
    DROP TABLE IF EXISTS test_percentile_disc;
    CREATE TEMP TABLE test_percentile_disc AS
        (SELECT percentile_disc(0.2) within GROUP (ORDER BY __samp_out_label) AS __samp_out_label
        FROM temp_nyc_label);

    DROP TABLE IF EXISTS nyc_train;
    CREATE TABLE nyc_train AS
        (SELECT temp_nyc_label.*
            FROM temp_nyc_label
            INNER JOIN train_percentile_disc
            ON temp_nyc_label.__samp_out_label <= train_percentile_disc.__samp_out_label
        );
    DROP TABLE IF EXISTS nyc_test;
    CREATE TABLE nyc_test AS
        (SELECT temp_nyc_label.*
            FROM temp_nyc_label
            INNER JOIN test_percentile_disc
            ON temp_nyc_label.__samp_out_label <= test_percentile_disc.__samp_out_label
        );""", has_results=False
)

In [22]:
nyc_train = db.open_table("nyc_train")
nyc_test = db.open_table("nyc_test")

## Data Modeling

### Model Training
To begin, we want to build a linear regression model to predict the trip duration of NYC taxi using data `Table` in nyc_train.

In [23]:
from typing import List

# CREATE TYPE linreg_type AS (
#    col_nm text[]
#    , coef float8[]
#    , intercept float8
#    , serialized_linreg_model bytea
# );


class LinregType:
    col_nm: List[str]
    coef: List[float]
    intercept: float
    serialized_linreg_model: bytes


# -- Create function
# -- Need to specify the return type -> API will create the corresponding type in Greenplum to return a row
# -- Will add argument to change language extensions, currently plpython3u by default

@gp.create_array_function
def linreg_func(vendor_id: List[float], passenger_count: List[float], store_and_fwd_flag: List[float], day_of_week: List[float],hour_of_the_day: List[float], month: List[float], distance: List[float], trip_duration: List[float]) -> LinregType:

    import numpy as np
    col_nm = ["vendor_id", "passenger_count", "store_and_fwd_flag", "day_of_week", "hour_of_the_day", "month", "distance"]
    X=np.array([vendor_id, passenger_count, store_and_fwd_flag, day_of_week, hour_of_the_day, month, distance]).T
    y=np.array([trip_duration]).T

    from sklearn.linear_model import LinearRegression

    # Fit the logistic regression model
    lr_fit = LinearRegression().fit(X, y)

    lr_coef = lr_fit.coef_
    lr_intercept = lr_fit.intercept_

    # Serialization of the fitted model
    import six
    pickle = six.moves.cPickle
    lr_fit_model = pickle.dumps(lr_fit, protocol = 3)

    return {
        'col_nm': col_nm,
        'coef' : lr_coef[0],
        'intercept' : lr_intercept[0],
        'serialized_linreg_model' : lr_fit_model
    }

We apply our training function on our training data and obtain a new `Table' named linreg_fitted where stocked trained model.

In [24]:
linreg_fitted = (
    nyc_train.apply(lambda t: linreg_func(
                                t["vendor_id"],
                                t["passenger_count"],
                                t["store_and_fwd_flag_int"],
                                t["day_of_week_int"],
                                t["hour_of_the_day"],
                                t["month"],
                                t["distance"],
                                t["trip_duration"]
                                ),
    expand=True,
    )
) # Obtain the trained Linear Regression Model

In [25]:
linreg_fitted[["col_nm", "coef", "intercept"]] # Observe the model

col_nm,coef,intercept
"['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'day_of_week', 'hour_of_the_day', 'month', 'distance']","[192.0847403027395, 7.822347337390298, 53.68572350116058, -2.3280204868372127, 4.602986436408932, 16.094583774740485, 116.43566563401572]",136.98037611697077


### Prediction
After training, we want to predict our test set, that's why we will need a predict function,depending on the inout values, it will return the trip duration computed by the trained model.

In [26]:
@gp.create_function
def linreg_pred_func(serialized_model: bytes, vendor_id: int, passenger_count: int, store_and_fwd_flag: int, day_of_week: int,hour_of_the_day: float, month: float, distance: float) -> float:
    # Deserialize the serialized model
    import six

    pickle = six.moves.cPickle
    model = pickle.loads(serialized_model)

    features = [vendor_id, passenger_count, store_and_fwd_flag, day_of_week,
                hour_of_the_day, month, distance]

    # Predict the target variable
    y_pred = model.predict([features])

    return y_pred[0][0]

But before apply it on our test set, since test set and model are saving in two different `Table`, we need firstly join two table together.

In [27]:
linreg_test_fit = linreg_fitted.cross_join(
    nyc_test,
    self_columns=["col_nm", "coef", "intercept", "serialized_linreg_model"],
    other_columns=["*"],
) # Since GreenplumPython can apply function on one table, we need to join model table and test set table

In [28]:
linreg_test_fit[:3]

col_nm,coef,intercept,serialized_linreg_model,vendor_id,passenger_count,store_and_fwd_flag_int,trip_duration,day_of_week_int,hour_of_the_day,month,distance,__samp_out_label
"['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'day_of_week', 'hour_of_the_day', 'month', 'distance']","[192.08474030924384, 7.822347337344347, 53.68572349958539, -2.328020486832102, 4.602986436420854, 16.09458377474276, 116.43566563401335]",136.98037610688687,\x800363736b6c6561726e2e6c696e6561725f6d6f64656c2e5f626173650a4c696e65617252656772657373696f6e0a7100298171017d710228580d0000006669745f696e7465726365707471038858090000006e6f726d616c697a657104580a0000006465707265636174656471055806000000636f70795f5871068858060000006e5f6a6f627371074e5808000000706f736974697665710889580e0000006e5f66656174757265735f696e5f71094b075805000000636f65665f710a636e756d70792e636f72652e6d756c746961727261790a5f7265636f6e7374727563740a710b636e756d70790a6e6461727261790a710c4b0085710d430162710e87710f527110284b014b014b07867111636e756d70790a64747970650a71125802000000663871138988877114527115284b0358010000003c71164e4e4e4affffffff4affffffff4b00747117628943385e1b4f31b6026840f8629f6b154a1f40b268a2c9c5d74a4020e57a34c99f02c0c56fc146756912409c406ba43618304005851cf2e11b5d40711874711962580500000072616e6b5f711a4b07580900000073696e67756c61725f711b680b680c4b0085711c680e87711d52711e284b014b0785711f6815894338e251371cc102bb4099c48bf9cb00b2405107d5453670a0405397880b245b9c408f4637b7dc539640697299c71d048040ef49be6c62e35340712074712162580a000000696e746572636570745f7122680b680c4b00857123680e877124527125284b014b0185712668158943087c9bb63d5f1f614071277471286258100000005f736b6c6561726e5f76657273696f6e71295805000000312e312e32712a75622e,2,1,0,1274,2,22,5,3.80613939487619,0.10143846247242294
"['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'day_of_week', 'hour_of_the_day', 'month', 'distance']","[192.08474030924384, 7.822347337344347, 53.68572349958539, -2.328020486832102, 4.602986436420854, 16.09458377474276, 116.43566563401335]",136.98037610688687,\x800363736b6c6561726e2e6c696e6561725f6d6f64656c2e5f626173650a4c696e65617252656772657373696f6e0a7100298171017d710228580d0000006669745f696e7465726365707471038858090000006e6f726d616c697a657104580a0000006465707265636174656471055806000000636f70795f5871068858060000006e5f6a6f627371074e5808000000706f736974697665710889580e0000006e5f66656174757265735f696e5f71094b075805000000636f65665f710a636e756d70792e636f72652e6d756c746961727261790a5f7265636f6e7374727563740a710b636e756d70790a6e6461727261790a710c4b0085710d430162710e87710f527110284b014b014b07867111636e756d70790a64747970650a71125802000000663871138988877114527115284b0358010000003c71164e4e4e4affffffff4affffffff4b00747117628943385e1b4f31b6026840f8629f6b154a1f40b268a2c9c5d74a4020e57a34c99f02c0c56fc146756912409c406ba43618304005851cf2e11b5d40711874711962580500000072616e6b5f711a4b07580900000073696e67756c61725f711b680b680c4b0085711c680e87711d52711e284b014b0785711f6815894338e251371cc102bb4099c48bf9cb00b2405107d5453670a0405397880b245b9c408f4637b7dc539640697299c71d048040ef49be6c62e35340712074712162580a000000696e746572636570745f7122680b680c4b00857123680e877124527125284b014b0185712668158943087c9bb63d5f1f614071277471286258100000005f736b6c6561726e5f76657273696f6e71295805000000312e312e32712a75622e,2,2,0,900,6,15,1,3.3173723813679055,0.032740062705734374
"['vendor_id', 'passenger_count', 'store_and_fwd_flag', 'day_of_week', 'hour_of_the_day', 'month', 'distance']","[192.08474030924384, 7.822347337344347, 53.68572349958539, -2.328020486832102, 4.602986436420854, 16.09458377474276, 116.43566563401335]",136.98037610688687,\x800363736b6c6561726e2e6c696e6561725f6d6f64656c2e5f626173650a4c696e65617252656772657373696f6e0a7100298171017d710228580d0000006669745f696e7465726365707471038858090000006e6f726d616c697a657104580a0000006465707265636174656471055806000000636f70795f5871068858060000006e5f6a6f627371074e5808000000706f736974697665710889580e0000006e5f66656174757265735f696e5f71094b075805000000636f65665f710a636e756d70792e636f72652e6d756c74696172726

Now, we can begin the prediction

In [29]:
linreg_pred = linreg_test_fit.assign(
    pred=lambda t: linreg_pred_func(
            t["serialized_linreg_model"],
            t["vendor_id"],
            t["passenger_count"],
            t["store_and_fwd_flag_int"],
            t["day_of_week_int"],
            t["hour_of_the_day"],
            t["month"],
            t["distance"],
        ),
)[["trip_duration", "pred"]] # Predict test set

In [30]:
linreg_pred[:5]

trip_duration,pred
1274,1149.2251575022772
900,994.2262701805902
337,782.9945828055118
913,1175.0894969524852
138,700.7955972495773


## Model Evaluation

It will be difficult to evaluate performance of our model by observing data one by one, so we need to create a function to compare the real trip duration and trip duration computed by model.

In [31]:
class linreg_eval_type:
    mae: float
    mape: float
    mse: float
    r2_score: float

In [32]:
@gp.create_array_function
def linreg_eval(y_actual: List[float], y_pred: List[float]) -> linreg_eval_type:
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import numpy as np

    mae = mean_absolute_error(y_actual, y_pred)
    mse = mean_squared_error(y_actual, y_pred)
    r2_score = r2_score(y_actual, y_pred)

    y_pred_f = np.array(y_pred, dtype=float)
    mape = 100 * sum(abs(y_actual - y_pred_f) / y_actual) / len(y_actual)

    # eval_score = [mae, mape, mse, r2_score]

    return {"mae": mae, "mape": mape, "mse": mse, "r2_score": r2_score}

In [33]:
linreg_pred.apply(
    lambda t: linreg_eval(t["trip_duration"], t["pred"]), expand=True)

mae,mape,mse,r2_score
470.2574016869885,96.6737826759858,66430955.50738679,0.004744146807444949


Our model is not very good in terms of accuracy and error, need to find another model which can improve accuracy, for example XGboost.